# Introduction à Langchain 



### Dépendances et clé d'API

In [1]:
#Décommenter sur Google Colab
#%pip install langchain langchain-openai langchain_mistralai openai python-dotenv -q
#from google.colab import userdata
#api_key=userdata.get('OPENAI_API_KEY')


#Décommenter en local
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key= getenv("OPENAI_API_KEY")

### Le modèle de LLM

Basiquement Langchain nécessite simplement d'instancier un `Runnable` pour le modèle de LLM et de l'invoquer directement

Langchain fournit une intégration pour un grand nombre de [providers et de modèles](https://python.langchain.com/v0.2/docs/integrations/chat/). Elles partagent une interface commune, permettant de switcher facilement. Attention, tous les modèles ne supportent pas les mêmes fonctionnalités (streaming, structure json, usage de token, etc.)

<img src="./assets/runnable_model.png" alt="drawing" width="400"/>

In [3]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(model="gpt-4o-mini", api_key=getenv('OPENAI_API_KEY'))

# Approche générique équivalente
from langchain.chat_models import init_chat_model
llm_openai = init_chat_model("gpt-4o-mini", model_provider="openai", api_key=getenv('OPENAI_API_KEY'))

On peut invoquer le runnable (et donc le model directement).
C'est une abstraction au dessus de `client.chat.completions.create`

In [4]:
response_4o_mini = llm_openai.invoke("Hello, how are you today?")
response_4o_mini

AIMessage(content='Hello! I’m just a program, so I don’t have feelings, but I’m here and ready to assist you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 14, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bgt0zLoSfECsmi6wocGCTFFQPwFFA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--88e76b27-c4f8-4ddc-a281-e88a45a4b73b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 32, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

La complétion consiste en un objet `AIMessage`
* **content** - la complétion
* **response_metadata** - Metadata additionnels. Par exemple: response headers, logprobs, token counts.

In [5]:
print(response_4o_mini.content)
print(response_4o_mini.response_metadata)

Hello! I’m just a program, so I don’t have feelings, but I’m here and ready to assist you. How can I help you today?
{'token_usage': {'completion_tokens': 32, 'prompt_tokens': 14, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bgt0zLoSfECsmi6wocGCTFFQPwFFA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}


Pour appeler un autre modèle, par exemple un de Mistral on switch simplement le modèle créé. Tout reste identique.

In [6]:
from langchain_mistralai import ChatMistralAI

llm_mistral = ChatMistralAI(model="mistral-large-latest", api_key=getenv('MISTRAL_API_KEY'))
# Alternative générique équivalente
# llm_mistral = init_chat_model("mistral-large-latest", model_provider="mostralai", api_key=getenv('MISTRAL_API_KEY'))

response_mistral = llm_mistral.invoke("Hello, how are you today?")



response_mistral

AIMessage(content="Hello! I'm functioning well, thank you. How about you? How's your day going?", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 10, 'total_tokens': 33, 'completion_tokens': 23}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run--001f3a70-a511-4f28-b8e6-8ed15334e361-0', usage_metadata={'input_tokens': 10, 'output_tokens': 23, 'total_tokens': 33})

In [7]:
print(response_mistral.content)
print(response_mistral.response_metadata)

Hello! I'm functioning well, thank you. How about you? How's your day going?
{'token_usage': {'prompt_tokens': 10, 'total_tokens': 33, 'completion_tokens': 23}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}


### Le prompt

Nous avons invoqué le modèle en lui passant directement le prompt comme `string`. Très souvent, et encore plus lorsque l'on voudra *augmenter* le modèle, notre prompt n'est pas une chaîne en dure mais composée de plusieurs partie, certaines potentiellement variables.

Typiquement le prompt se décompose de deux parties
* La partie `system` ou `preprompt`, qui peut par exemple contenir des instructions pour le modèle (par exemple de formattage de la réponse),  
* La partie `user` contenant la requête de l'utilisateur à proprement parlé.

In [8]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Répond en turc."), #Ici des instructions pour le modèle, 'à l'insu' de l'utilisateur
    ("human", "{question}"), # Ici la question de l'utilisateur, variable {question} à remplacer sera remplacée par la question de l'utilisateur
])

# Approche sans rôles via PromptTemplate
prompt_2 = PromptTemplate.from_template(
    "Répond en turc à la question suivante : {question}"
)


Un fois le prompt créé, on peut l'envoyer/le piper au modèle pour créer une chaîne de runnables `prompt -> model`.

In [9]:
chain = prompt | llm_openai
# La chaîne est invoquée avec un dictionnaire contenant les valeurs pour les variables du prompt
response = chain.invoke({"question": "What is the capital of France?"})

response.content

"Fransa'nın başkenti Paris'tir."